In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [59]:
df = pd.read_pickle('2015_fires.pkl')

In [60]:
natural = []

for idx, row in df.iterrows():
    if row[0] == 'Lightning':
        natural.append(1)
    else:
        natural.append(0)

df['natural_cause'] = natural

In [61]:
df.head()

,STAT_CAUSE_DESCR,LATITUDE,LONGITUDE,STATE,DISCOVERY_DATE,FIRE_SIZE,DATE,MONTH,YEAR,DAY_OF_WEEK,MTHYR,tmax_5,tmin_5,prcp_5,snow_5,last_7,last_14,last_28,natural_cause
0,Campfire,60.910278,-149.458611,AK,2457162.5,0.10,2015-05-20,5,2015,Wednesday,2015-5,15.68,3.20,0.0,0.0,0.000000,165.071429,98.750000,0
1,Campfire,60.507778,-149.811111,AK,2457173.5,0.10,2015-05-31,5,2015,Sunday,2015-5,25.78,5.54,0.0,0.0,0.714286,3.071429,23.500000,0
2,Campfire,60.435000,-149.343333,AK,2457220.5,0.10,2015-07-17,7,2015,Friday,2015-7,15.28,8.52,33.0,0.0,139.857143,77.714286,62.785714,0
3,Miscellaneous,60.399722,-149.643889,AK,2457214.5,0.17,2015-07-11,7,2015,Saturday,2015-7,18.80,7.96,13.2,0.0,10.571429,25.285714,13.928571,0
4,Campfire,60.814444,-149.536389,AK,2457164.5,61.60,2015-05-22,5,2015,Friday,2015-5,20.86,3.24,0.0,0.0,0.000000,72.500000,75.107143,0


In [62]:
# df = df[df['FIRE_SIZE'] > 100]

In [63]:
print(df.corr(method='pearson'))

                LATITUDE  LONGITUDE  DISCOVERY_DATE  FIRE_SIZE     MONTH  \
LATITUDE        1.000000  -0.069004        0.053674   0.180194  0.047325   
LONGITUDE      -0.069004   1.000000       -0.155465  -0.181578 -0.108462   
DISCOVERY_DATE  0.053674  -0.155465        1.000000   0.051217  0.972592   
FIRE_SIZE       0.180194  -0.181578        0.051217   1.000000  0.029954   
MONTH           0.047325  -0.108462        0.972592   0.029954  1.000000   
YEAR                 NaN        NaN             NaN        NaN       NaN   
tmax_5          0.312054  -0.174694        0.257351   0.246268  0.183291   
tmin_5          0.228816  -0.335130        0.394323   0.253865  0.321011   
prcp_5         -0.077918   0.124300        0.116672  -0.030498  0.112837   
snow_5         -0.026204   0.011576       -0.082855  -0.011229 -0.072758   
last_7         -0.282977   0.301591        0.179484  -0.105696  0.201399   
last_14        -0.347245   0.337251        0.165891  -0.091890  0.181721   
last_28     

In [64]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import ExtraTreesRegressor

In [65]:
X = pd.concat([df.iloc[:,7], df.iloc[:,11:]], axis=1)
Y = df.iloc[:,5]              

In [66]:
int(len(df)*.2)

154

In [67]:
X_train = X.iloc[154:]
Y_train = Y[154:]

X_test = X.iloc[:154]
Y_test = Y[:154]

In [79]:
etr = ExtraTreesRegressor()
rfe = RFE(etr, 5)
fit = rfe.fit(X_train, Y_train)

In [80]:
fit.ranking_

array([4, 1, 1, 3, 5, 1, 1, 1, 2])